In [1]:
import html
import re

import tensorflow_datasets as tfds

class SubalphabetSubwordTextEncoder:
    HTML_ESCAPE_PATTERN = re.compile('&(((#\d+)|([A-Za-z]+));&?)+')
    SPECIAL_CHAR = 'X'

    def __init__(self, subalphabet, vocab_list=None):
        self._subword_text_encoder = tfds.features.text.SubwordTextEncoder(vocab_list)
        self._subalphabet = subalphabet

    def encode(self, s):
        subword_max_code = len(self.subwords)
        codes = self._build_encoding_dict(self._subalphabet)
        preprocessed_string = self.preprocess_string(s)

        result = []
        unicode_buffer = bytearray()
        for id in self._subword_text_encoder.encode(preprocessed_string):
            if id <= subword_max_code:
                result.append(id)
            else:
                unicode_buffer.append(id - subword_max_code - 1)
                try:
                    id = ord(unicode_buffer.decode())
                    result.append(subword_max_code + codes[id] + 1)
                    unicode_buffer = bytearray()
                except UnicodeDecodeError:
                    pass
        return result

    def decode(self, ids):
        subword_max_code = len(self.subwords)

        processed_ids = []
        for id in ids:
            if id <= subword_max_code:
                processed_ids.append(id)
            else:
                char = self._subalphabet[id - subword_max_code - 1].encode()
                processed_ids += [codepoint + subword_max_code + 1 for codepoint in char]

        return self._subword_text_encoder.decode(processed_ids)
    
    @classmethod
    def build_from_corpus(cls, corpus_generator, subalphabet_size, target_vocab_size, max_subword_length=20, max_corpus_chars=None, reserved_tokens=None):
        simplified_strings = []
        char_counts = defaultdict(lambda: 0)

        for string in corpus_generator:
            simplified_string = cls._unescape_string(string.lower())
            simplified_strings.append(simplified_string)
            for char in simplified_string:
                char_counts[char] += 1

        sorted_char_counts = sorted(char_counts.items(), key = lambda item: item[1], reverse = True)
        chars = [char for char, char_count in sorted_char_counts[:subalphabet_size - 1]]
        chars = list(reversed(chars)) # We reverse the chars so that the most common ones are last
        chars = cls._ensure_special_character_presence(chars)

        def simplified_corpus():
            for string in simplified_strings:
                yield cls._compact_string(chars, string)

        subword_text_encoder = tfds.features.text.SubwordTextEncoder.build_from_corpus(simplified_corpus(), target_vocab_size + 256 - subalphabet_size)
        
        return cls(chars, subword_text_encoder.subwords)
    
    @classmethod
    def load_from_file(cls, filename_prefix):
        subword_text_encoder = tfds.features.text.SubwordTextEncoder.load_from_file(filename_prefix)
        with open(filename_prefix + '.subalphabet', 'rb') as text_file:
            subalphabet = ['\0', cls.SPECIAL_CHAR] + text_file.read().decode().split('\0')

        return cls(subalphabet, subword_text_encoder.subwords)
    
    def save_to_file(self, filename_prefix):
        self._subword_text_encoder.save_to_file(filename_prefix)
        with open(filename_prefix + '.subalphabet', 'wb') as text_file:
            text_file.write('\0'.join(self._subalphabet[2:]).encode())

    def preprocess_string(self, string):
        return self._compact_string(self._subalphabet, self._unescape_string(string.lower()))

    @property
    def vocab_size(self):
        return self._subword_text_encoder.vocab_size - 256 + len(self._subalphabet)
    
    @property
    def subwords(self):
        return self._subword_text_encoder.subwords

    @classmethod
    def _ensure_special_character_presence(cls, chars):
        if '\0' in chars:
            chars = [char for char in chars if char != '\0']
        else:
            chars = chars[1:]

        return ['\0', cls.SPECIAL_CHAR] + chars
    
    @classmethod
    def _unescape_html_symbol(cls, escaped):
        escaped_symbols = escaped.group(0).split(';')
        escaped_symbols = escaped_symbols[:-1] # strip the last string empty string
        escaped_symbols = (escaped_symbol + ';' for escaped_symbol in escaped_symbols)
        result = ''

        for escaped_symbol in escaped_symbols:
            code = escaped_symbol[1:] if escaped_symbol[0] == '&' else escaped_symbol

            if code[0] == '#':
                result += chr(int(code[1:-1]))
            elif code in html.entities.html5:
                result += html.entities.html5[code]
            else:
                result += escaped_symbol

        return result
    
    @classmethod
    def _unescape_string(cls, string):
        return cls.HTML_ESCAPE_PATTERN.sub(cls._unescape_html_symbol, string)
    
    @classmethod
    def _compact_string(cls, chars, string):
        if len(string) == 0:
            return string
        else:
            compacted_string = string[0]

            for char in string[1:]:
                if char not in chars or char == cls.SPECIAL_CHAR:
                    if compacted_string[-1] != cls.SPECIAL_CHAR:
                        compacted_string += cls.SPECIAL_CHAR
                else:
                    compacted_string += char

            return compacted_string
    
    @classmethod
    def _build_encoding_dict(cls, chars):
        return {ord(char): index for index, char in enumerate(chars)}

In [2]:
encoder = SubalphabetSubwordTextEncoder.load_from_file('72_256')

In [3]:
import itertools
import math
import os

import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds

physical_devices = tf.config.list_physical_devices('GPU')
for physical_device in physical_devices:
    tf.config.experimental.set_memory_growth(physical_device, enable=True)

In [4]:
TYPE=np.int16

class Articles:
    EMPTY_ARTICLE = np.array([], dtype=TYPE) # used for padding
    
    def __init__(self, path):
        with open(path, 'rb') as text_file:
            data = text_file.read().decode()

        self.articles = sorted(set(data.split('\0')[:20000]), key=len)
        self._encoded_articles = None

    @property
    def encoded_articles(self):
        if self._encoded_articles == None:
            self._encoded_articles = [np.array(encoder.encode(article), dtype=TYPE) for article in self.articles]
        
        return self._encoded_articles

    def articles_generator(self, batch_size = 1, start = 0, end = None):
        end = end or len(self.articles)

        for _ in range(batch_size - ((end - start - 1) % batch_size + 1)):
            yield self.EMPTY_ARTICLE

        for article in itertools.islice(self.encoded_articles, start, end):
            yield article

    def subbatch_generator(self, batch_size, batch_length, start = 0, end = None):
        end = end or len(self.articles)

        dataset = tf.data.Dataset.from_generator(self.articles_generator, args=(batch_size, start, end), output_types=TYPE)
        dataset = dataset.padded_batch(batch_size, padded_shapes=([None]), drop_remainder=True)
        dataset = dataset.shuffle(100)

        for batch in dataset.as_numpy_iterator():
            remaining = batch
            while remaining.shape[1] > batch_length + 1:
                yield remaining[:, :batch_length + 1]
                remaining = remaining[:, batch_length:]

            if remaining.shape[1] == batch_length + 1:
                yield remaining
                yield np.zeros((batch_size, batch_length + 1), dtype=TYPE)
            else:
                yield np.hstack([remaining, np.zeros([batch_size, batch_length - remaining.shape[1] + 1])])

    def steps(self, batch_size, batch_length):
        articles = self.articles_generator(batch_size, batch_length)
        return sum(math.ceil(len(article) / batch_length + 1) for i, article in enumerate(articles) if (i + 1) % batch_size == 0)

    def dataset(self, batch_size, batch_length, start = 0, end = None):
        end = end or len(self.articles)

        dataset = tf.data.Dataset.from_generator(self.subbatch_generator, args=(batch_size, batch_length, start, end), output_types=TYPE, output_shapes=(batch_size, batch_length + 1))
        return dataset.map(lambda batch: (batch[:, :-1], batch[:, 1:]))

In [5]:
def loss(labels, logits):
    return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

def average_final_batch_ratio(true_labels, predictions):
    return 0 ** tf.math.abs(true_labels[-1, -1])

class ModelStateResetter(tf.keras.callbacks.Callback):
    def __init__(self):
        pass

    def on_epoch_begin(self, epoch, logs=None):
        self.last_final_batch_count = 0
        
    def on_batch_end(self, batch, logs={}):
        average_final_batch_ratio = logs.get('average_final_batch_ratio', 0)
        final_batch_count = int(round(average_final_batch_ratio * (batch + 1)))
        is_final = final_batch_count - self.last_final_batch_count
        self.last_final_batch_count = final_batch_count
        
        if is_final:
            self.model.reset_states()

class Model:
    def __init__(self, articles, checkpoint_dir, vocab_size, embedding_dim, rnn_units):
        self._articles = articles
        self._batch_size = None
        self._batched_item_length = None
        self._training_model = None
        self._predicting_model = None
        self._vocab_size = vocab_size
        self._embedding_dim = embedding_dim
        self._rnn_units = rnn_units

        self._checkpoint_dir = checkpoint_dir
        self._checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}") # Name of the checkpoint files

    def training_model(self, batch_size, batched_item_length):
        if self._training_model == None or batch_size != self._batch_size or batched_item_length != self._batched_item_length:
            self._batch_size = batch_size
            self._batched_item_length = batched_item_length
            self._training_model = tf.keras.Sequential([
                tf.keras.layers.Masking(mask_value=0, batch_input_shape=[batch_size, batched_item_length]),
                tf.keras.layers.Embedding(self._vocab_size, self._embedding_dim),
                tf.keras.layers.GRU(self._rnn_units, return_sequences=True, stateful=True, recurrent_initializer='glorot_uniform'),
                tf.keras.layers.GRU(self._rnn_units, return_sequences=True, stateful=True, recurrent_initializer='glorot_uniform'),
                tf.keras.layers.GRU(self._rnn_units, return_sequences=True, stateful=True, recurrent_initializer='glorot_uniform'),
                tf.keras.layers.GRU(self._rnn_units, return_sequences=True, stateful=True, recurrent_initializer='glorot_uniform'),
                tf.keras.layers.Dense(self._vocab_size),
            ])

            if os.path.isdir(self._checkpoint_dir):
                self._training_model.load_weights(tf.train.latest_checkpoint(self._checkpoint_dir))

            self._training_model.compile(optimizer='adam', loss=loss, metrics=[average_final_batch_ratio])
            self._predicting_model = None
        
        return self._training_model

    @property
    def callbacks(self):
        checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(filepath=self._checkpoint_prefix, save_weights_only=True)
        model_state_resetter_callback = ModelStateResetter()
        
        return [checkpoint_callback, model_state_resetter_callback]
    
    def train(self, batch_size, batched_item_length, epochs=1):
        dataset = self._articles.dataset(batch_size, batched_item_length)

        model = self.training_model(batch_size, batched_item_length)

        model.fit(dataset, epochs=epochs, callbacks=self.callbacks)
    
    @property
    def predicting_model(self):
        if self._predicting_model == None:
            self._predicting_model = tf.keras.Sequential([
                tf.keras.layers.Masking(mask_value=0, batch_input_shape=[1, 1]),
                tf.keras.layers.Embedding(self._vocab_size, self._embedding_dim),
                tf.keras.layers.GRU(self._rnn_units, stateful=True, return_sequences=True),
                tf.keras.layers.GRU(self._rnn_units, stateful=True, return_sequences=True),
                tf.keras.layers.GRU(self._rnn_units, stateful=True, return_sequences=True),
                tf.keras.layers.GRU(self._rnn_units, stateful=True, return_sequences=True),
                tf.keras.layers.Dense(self._vocab_size),
            ])
            
            self._predicting_model.load_weights(tf.train.latest_checkpoint(self._checkpoint_dir))
            self._training_model = None
        
        return self._predicting_model
    
    def predict(self, input_eval):
        return self.predicting_model(input_eval)

In [6]:
articles = Articles('page_revisions_text_with_title')

In [7]:
model = Model(articles, './training_checkpoints-26', vocab_size = encoder.vocab_size, embedding_dim=64, rnn_units=512)

In [8]:
model.training_model(256, 256).summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
masking (Masking)            (256, 256)                0         
_________________________________________________________________
embedding (Embedding)        (256, 256, 64)            16320     
_________________________________________________________________
gru (GRU)                    (256, 256, 512)           887808    
_________________________________________________________________
gru_1 (GRU)                  (256, 256, 512)           1575936   
_________________________________________________________________
gru_2 (GRU)                  (256, 256, 512)           1575936   
_________________________________________________________________
gru_3 (GRU)                  (256, 256, 512)           1575936   
_________________________________________________________________
dense (Dense)                (256, 256, 255)           1

In [9]:
model.train(256, 256, epochs=30)

Epoch 1/30
1902/1902 [==============================] - 1116s 587ms/step - loss: 2.6060 - average_final_batch_ratio: 0.1646
Epoch 2/30
1902/1902 [==============================] - 895s 471ms/step - loss: 1.8007 - average_final_batch_ratio: 0.1646
Epoch 3/30
1902/1902 [==============================] - 901s 474ms/step - loss: 1.6343 - average_final_batch_ratio: 0.1646
Epoch 4/30
1902/1902 [==============================] - 892s 469ms/step - loss: 1.5545 - average_final_batch_ratio: 0.1646
Epoch 5/30
1902/1902 [==============================] - 898s 472ms/step - loss: 1.5133 - average_final_batch_ratio: 0.1646
Epoch 6/30
1902/1902 [==============================] - 901s 474ms/step - loss: 1.4781 - average_final_batch_ratio: 0.1646
Epoch 7/30
1902/1902 [==============================] - 885s 465ms/step - loss: 1.4568 - average_final_batch_ratio: 0.1646
Epoch 8/30
1902/1902 [==============================] - 918s 483ms/step - loss: 1.4347 - average_final_batch_ratio: 0.1646
Epoch 9/30
1902

In [10]:
model.train(256, 256, epochs=10)

Epoch 1/10
1902/1902 [==============================] - 906s 476ms/step - loss: 1.3793 - average_final_batch_ratio: 0.1646
Epoch 2/10
1902/1902 [==============================] - 905s 476ms/step - loss: 1.4925 - average_final_batch_ratio: 0.1646
Epoch 3/10
1902/1902 [==============================] - 907s 477ms/step - loss: 1.4661 - average_final_batch_ratio: 0.1646
Epoch 4/10
1902/1902 [==============================] - 911s 479ms/step - loss: 1.4477 - average_final_batch_ratio: 0.1646
Epoch 5/10
1902/1902 [==============================] - 894s 470ms/step - loss: 1.4336 - average_final_batch_ratio: 0.1646
Epoch 6/10
1902/1902 [==============================] - 912s 480ms/step - loss: 1.4201 - average_final_batch_ratio: 0.1646
Epoch 7/10
1902/1902 [==============================] - 911s 479ms/step - loss: 1.4068 - average_final_batch_ratio: 0.1646
Epoch 8/10
1902/1902 [==============================] - 919s 483ms/step - loss: 1.3954 - average_final_batch_ratio: 0.1646
Epoch 9/10
1902/

In [11]:
import ctypes

class Huffman:
    huffman = ctypes.CDLL('x64/Release/huffman')
    
    huffman.create_tree.restype = ctypes.c_void_p
    huffman.destroy_tree.restype = None
    huffman.load_weights.restype = None
    huffman.create_code_string.restype = ctypes.c_char_p
    
    def __init__(self, category_count):
        self.category_count = category_count
        self.tree = ctypes.c_void_p(self.huffman.create_tree(category_count))

    def __del__(self):
        self.huffman.destroy_tree(self.tree)
        
    def load_weights(self, weights):
        self.huffman.load_weights(self.tree, weights.ctypes.data_as(ctypes.POINTER(ctypes.c_float)))
    
    def get_code_length(self, category):
        return self.huffman.get_code_length(self.tree, category)

    def get_code_zero_count(self, category):
        return self.huffman.get_code_zero_count(self.tree, category)
    
    def archive_size(self, model, text):
        archived_size = math.ceil(math.log2(self.category_count))
        input_eval = np.array([[text[0]]], dtype=TYPE)

        model.predicting_model.reset_states()

        for byte in text[1:]:
            predictions = model.predict(input_eval)
            predictions = tf.squeeze(predictions, 0) # remove the batch dimension

            weights = tf.nn.softmax(predictions[0]).numpy()
            self.load_weights(weights)
            archived_size += self.get_code_length(byte.item())

            input_eval = tf.expand_dims([byte], 0)

        return archived_size

In [15]:
total_raw = 0
total_compressed = 0

huffman = Huffman(encoder.vocab_size)
for index, encoded_article in enumerate(articles.articles_generator(1)):
    if index % 100 == 0:
        article = encoder.decode(encoded_article)
        if len(article) == 0:
            continue
        total_raw += len(article) * 8
        total_compressed += huffman.archive_size(model, encoded_article)
        print('Article %d:\tLength: %d\tAvg Compression: %f' % (index, len(article), total_compressed/total_raw))

Article 0:	Length: 12	Avg Compression: 0.572917
Article 100:	Length: 28	Avg Compression: 0.590625
Article 200:	Length: 30	Avg Compression: 0.580357
Article 300:	Length: 31	Avg Compression: 0.606436
Article 400:	Length: 32	Avg Compression: 0.612782
Article 500:	Length: 33	Avg Compression: 0.608434
Article 600:	Length: 34	Avg Compression: 0.613750
Article 700:	Length: 35	Avg Compression: 0.608511
Article 800:	Length: 36	Avg Compression: 0.613930
Article 900:	Length: 37	Avg Compression: 0.602679
Article 1000:	Length: 38	Avg Compression: 0.600072
Article 1100:	Length: 39	Avg Compression: 0.599026
Article 1200:	Length: 39	Avg Compression: 0.600236
Article 1300:	Length: 40	Avg Compression: 0.599946
Article 1400:	Length: 41	Avg Compression: 0.594802
Article 1500:	Length: 41	Avg Compression: 0.599359
Article 1600:	Length: 42	Avg Compression: 0.602466
Article 1700:	Length: 42	Avg Compression: 0.599405
Article 1800:	Length: 43	Avg Compression: 0.603269
Article 1900:	Length: 44	Avg Compression: 0

Article 15700:	Length: 11993	Avg Compression: 0.520490
Article 15800:	Length: 12274	Avg Compression: 0.519436
Article 15900:	Length: 12522	Avg Compression: 0.519417
Article 16000:	Length: 12843	Avg Compression: 0.519216
Article 16100:	Length: 13022	Avg Compression: 0.519412
Article 16200:	Length: 13335	Avg Compression: 0.520001
Article 16300:	Length: 13582	Avg Compression: 0.520906
Article 16400:	Length: 13943	Avg Compression: 0.520872
Article 16500:	Length: 14159	Avg Compression: 0.521914
Article 16600:	Length: 14497	Avg Compression: 0.523003
Article 16700:	Length: 14766	Avg Compression: 0.522815
Article 16800:	Length: 15138	Avg Compression: 0.522954
Article 16900:	Length: 15589	Avg Compression: 0.523196
Article 17000:	Length: 15906	Avg Compression: 0.523986
Article 17100:	Length: 16145	Avg Compression: 0.523922
Article 17200:	Length: 16798	Avg Compression: 0.524581
Article 17300:	Length: 17120	Avg Compression: 0.524358
Article 17400:	Length: 17810	Avg Compression: 0.525381
Article 17

In [11]:
model.train(256, 256, epochs=20)

Epoch 1/20
1902/1902 [==============================] - 907s 477ms/step - loss: 1.3621 - average_final_batch_ratio: 0.1646
Epoch 2/20
1902/1902 [==============================] - 888s 467ms/step - loss: 1.3554 - average_final_batch_ratio: 0.1646
Epoch 3/20
1902/1902 [==============================] - 906s 476ms/step - loss: 1.3571 - average_final_batch_ratio: 0.1646
Epoch 4/20
1902/1902 [==============================] - 908s 477ms/step - loss: 1.3426 - average_final_batch_ratio: 0.1646
Epoch 5/20
1902/1902 [==============================] - 904s 475ms/step - loss: 1.3330 - average_final_batch_ratio: 0.1646
Epoch 6/20
1902/1902 [==============================] - 915s 481ms/step - loss: 1.3277 - average_final_batch_ratio: 0.1646
Epoch 7/20
1902/1902 [==============================] - 929s 488ms/step - loss: 1.3222 - average_final_batch_ratio: 0.1646
Epoch 8/20
1902/1902 [==============================] - 922s 485ms/step - loss: 1.3163 - average_final_batch_ratio: 0.1646
Epoch 9/20
1902/

In [14]:
total_raw = 0
total_compressed = 0

huffman = Huffman(encoder.vocab_size)
for index, encoded_article in enumerate(articles.articles_generator(1)):
    if index % 100 == 0:
        article = encoder.decode(encoded_article)
        total_raw += len(article) * 8
        total_compressed += huffman.archive_size(model, encoded_article)
        print('Article %d:\tLength: %d\tAvg Compression: %f' % (index, len(article), total_compressed/total_raw))

Article 0:	Length: 12	Avg Compression: 0.218750
Article 100:	Length: 28	Avg Compression: 0.253125
Article 200:	Length: 30	Avg Compression: 0.282143
Article 300:	Length: 31	Avg Compression: 0.285891
Article 400:	Length: 32	Avg Compression: 0.266917
Article 500:	Length: 33	Avg Compression: 0.264307
Article 600:	Length: 34	Avg Compression: 0.268125
Article 700:	Length: 35	Avg Compression: 0.255851
Article 800:	Length: 36	Avg Compression: 0.252306
Article 900:	Length: 37	Avg Compression: 0.256899
Article 1000:	Length: 38	Avg Compression: 0.251445
Article 1100:	Length: 39	Avg Compression: 0.252273
Article 1200:	Length: 39	Avg Compression: 0.248821
Article 1300:	Length: 40	Avg Compression: 0.242726
Article 1400:	Length: 41	Avg Compression: 0.250248
Article 1500:	Length: 41	Avg Compression: 0.253663
Article 1600:	Length: 42	Avg Compression: 0.250850
Article 1700:	Length: 42	Avg Compression: 0.254762
Article 1800:	Length: 43	Avg Compression: 0.255572
Article 1900:	Length: 44	Avg Compression: 0

Article 15700:	Length: 11993	Avg Compression: 0.245166
Article 15800:	Length: 12274	Avg Compression: 0.244517
Article 15900:	Length: 12522	Avg Compression: 0.244238
Article 16000:	Length: 12843	Avg Compression: 0.243548
Article 16100:	Length: 13022	Avg Compression: 0.243688
Article 16200:	Length: 13335	Avg Compression: 0.244104
Article 16300:	Length: 13582	Avg Compression: 0.243245
Article 16400:	Length: 13868	Avg Compression: 0.243745
Article 16500:	Length: 14159	Avg Compression: 0.242819
Article 16600:	Length: 14497	Avg Compression: 0.241756
Article 16700:	Length: 14766	Avg Compression: 0.242192
Article 16800:	Length: 15189	Avg Compression: 0.242298
Article 16900:	Length: 15589	Avg Compression: 0.242662
Article 17000:	Length: 15922	Avg Compression: 0.241899
Article 17100:	Length: 16145	Avg Compression: 0.241635
Article 17200:	Length: 16798	Avg Compression: 0.240856
Article 17300:	Length: 17120	Avg Compression: 0.240251
Article 17400:	Length: 17810	Avg Compression: 0.240965
Article 17

In [15]:
model.train(256, 256, epochs=30)

Epoch 1/30
1902/1902 [==============================] - 889s 467ms/step - loss: 1.5974 - average_final_batch_ratio: 0.1646
Epoch 2/30
1902/1902 [==============================] - 885s 465ms/step - loss: 1.6071 - average_final_batch_ratio: 0.1646
Epoch 3/30
1902/1902 [==============================] - 883s 465ms/step - loss: 1.5557 - average_final_batch_ratio: 0.1646
Epoch 4/30
1902/1902 [==============================] - 895s 471ms/step - loss: 1.5194 - average_final_batch_ratio: 0.1646
Epoch 5/30
1902/1902 [==============================] - 883s 464ms/step - loss: 1.4900 - average_final_batch_ratio: 0.1646
Epoch 6/30
1902/1902 [==============================] - 898s 472ms/step - loss: 1.4650 - average_final_batch_ratio: 0.1646
Epoch 7/30
1902/1902 [==============================] - 891s 469ms/step - loss: 1.4420 - average_final_batch_ratio: 0.1646
Epoch 8/30
1902/1902 [==============================] - 889s 467ms/step - loss: 1.4234 - average_final_batch_ratio: 0.1646
Epoch 9/30
1902/

In [17]:
model.train(256, 256, epochs=10)

Epoch 1/10
1902/1902 [==============================] - 895s 471ms/step - loss: 1.7055 - average_final_batch_ratio: 0.1646
Epoch 2/10
1902/1902 [==============================] - 900s 473ms/step - loss: 1.6582 - average_final_batch_ratio: 0.1646
Epoch 3/10
1902/1902 [==============================] - 880s 463ms/step - loss: 1.8234 - average_final_batch_ratio: 0.1646
Epoch 4/10
1902/1902 [==============================] - 898s 472ms/step - loss: 1.6964 - average_final_batch_ratio: 0.1646
Epoch 5/10
1902/1902 [==============================] - 890s 468ms/step - loss: 1.6358 - average_final_batch_ratio: 0.1646
Epoch 6/10
1902/1902 [==============================] - 893s 470ms/step - loss: 1.5933 - average_final_batch_ratio: 0.1646
Epoch 7/10
1902/1902 [==============================] - 904s 475ms/step - loss: 1.5602 - average_final_batch_ratio: 0.1646
Epoch 8/10
1902/1902 [==============================] - 917s 482ms/step - loss: 1.5393 - average_final_batch_ratio: 0.1646
Epoch 9/10
1902/

In [18]:
total_raw = 0
total_compressed = 0

huffman = Huffman(encoder.vocab_size)
for index, encoded_article in enumerate(articles.articles_generator(1)):
    if index % 100 == 0:
        article = encoder.decode(encoded_article)
        total_raw += len(article) * 8
        total_compressed += huffman.archive_size(model, encoded_article)
        print('Article %d:\tLength: %d\tAvg Compression: %f' % (index, len(article), total_compressed/total_raw))

Article 0:	Length: 12	Avg Compression: 0.270833
Article 100:	Length: 28	Avg Compression: 0.281250
Article 200:	Length: 30	Avg Compression: 0.292857
Article 300:	Length: 31	Avg Compression: 0.303218
Article 400:	Length: 32	Avg Compression: 0.285714
Article 500:	Length: 33	Avg Compression: 0.288404
Article 600:	Length: 34	Avg Compression: 0.293125
Article 700:	Length: 35	Avg Compression: 0.279255
Article 800:	Length: 36	Avg Compression: 0.276753
Article 900:	Length: 37	Avg Compression: 0.274351
Article 1000:	Length: 38	Avg Compression: 0.269870
Article 1100:	Length: 39	Avg Compression: 0.267208
Article 1200:	Length: 39	Avg Compression: 0.263856
Article 1300:	Length: 40	Avg Compression: 0.257812
Article 1400:	Length: 41	Avg Compression: 0.265099
Article 1500:	Length: 41	Avg Compression: 0.264194
Article 1600:	Length: 42	Avg Compression: 0.260842
Article 1700:	Length: 42	Avg Compression: 0.266071
Article 1800:	Length: 43	Avg Compression: 0.269688
Article 1900:	Length: 44	Avg Compression: 0

Article 15700:	Length: 11993	Avg Compression: 0.248042
Article 15800:	Length: 12274	Avg Compression: 0.247397
Article 15900:	Length: 12522	Avg Compression: 0.247175
Article 16000:	Length: 12843	Avg Compression: 0.246457
Article 16100:	Length: 13022	Avg Compression: 0.246357
Article 16200:	Length: 13335	Avg Compression: 0.246753
Article 16300:	Length: 13582	Avg Compression: 0.246131
Article 16400:	Length: 13868	Avg Compression: 0.246546
Article 16500:	Length: 14159	Avg Compression: 0.245897
Article 16600:	Length: 14497	Avg Compression: 0.245093
Article 16700:	Length: 14766	Avg Compression: 0.245473
Article 16800:	Length: 15189	Avg Compression: 0.245603
Article 16900:	Length: 15589	Avg Compression: 0.245865
Article 17000:	Length: 15922	Avg Compression: 0.245350
Article 17100:	Length: 16145	Avg Compression: 0.245023
Article 17200:	Length: 16798	Avg Compression: 0.244476
Article 17300:	Length: 17120	Avg Compression: 0.243867
Article 17400:	Length: 17810	Avg Compression: 0.244692
Article 17

In [12]:
model.train(256, 256, epochs=10)

Epoch 1/10
1902/1902 [==============================] - 1135s 597ms/step - loss: 1.4816 - average_final_batch_ratio: 0.1667
Epoch 2/10
1902/1902 [==============================] - 897s 472ms/step - loss: 1.7947 - average_final_batch_ratio: 0.1667
Epoch 3/10
1902/1902 [==============================] - 915s 481ms/step - loss: 2.0383 - average_final_batch_ratio: 0.1667
Epoch 4/10
1902/1902 [==============================] - 900s 473ms/step - loss: 1.9310 - average_final_batch_ratio: 0.1667
Epoch 5/10
1902/1902 [==============================] - 912s 480ms/step - loss: 1.8722 - average_final_batch_ratio: 0.1667
Epoch 6/10
1902/1902 [==============================] - 906s 476ms/step - loss: 1.8283 - average_final_batch_ratio: 0.1667
Epoch 7/10
1902/1902 [==============================] - 894s 470ms/step - loss: 1.8167 - average_final_batch_ratio: 0.1667
Epoch 8/10
1902/1902 [==============================] - 907s 477ms/step - loss: 1.7877 - average_final_batch_ratio: 0.1667
Epoch 9/10
1902

In [14]:
total_raw = 0
total_compressed = 0

huffman = Huffman(encoder.vocab_size)
for index, encoded_article in enumerate(articles.articles_generator(1)):
    if index % 1000 == 0:
        article = encoder.decode(encoded_article)
        total_raw += len(article) * 8
        total_compressed += huffman.archive_size(model, encoded_article)
        print('Article %d:\tLength: %d\tAvg Compression: %f' % (index, len(article), total_compressed/total_raw))

Article 0:	Length: 12	Avg Compression: 0.260417
Article 1000:	Length: 38	Avg Compression: 0.310000
Article 2000:	Length: 44	Avg Compression: 0.272606
Article 3000:	Length: 50	Avg Compression: 0.309896
Article 4000:	Length: 56	Avg Compression: 0.316875
Article 5000:	Length: 67	Avg Compression: 0.333801
Article 6000:	Length: 293	Avg Compression: 0.286607
Article 7000:	Length: 880	Avg Compression: 0.268316
Article 8000:	Length: 1498	Avg Compression: 0.268465
Article 9000:	Length: 2197	Avg Compression: 0.274221
Article 10000:	Length: 2988	Avg Compression: 0.274560
Article 11000:	Length: 3898	Avg Compression: 0.268447
Article 12000:	Length: 4954	Avg Compression: 0.276038
Article 13000:	Length: 6351	Avg Compression: 0.272255
Article 14000:	Length: 8043	Avg Compression: 0.278312
Article 15000:	Length: 10150	Avg Compression: 0.282560
Article 16000:	Length: 12843	Avg Compression: 0.273624
Article 17000:	Length: 15922	Avg Compression: 0.267840
Article 18000:	Length: 20350	Avg Compression: 0.2665

In [15]:
model.train(256, 256, epochs=10)

Epoch 1/10
1902/1902 [==============================] - 928s 488ms/step - loss: 1.6736 - average_final_batch_ratio: 0.1667
Epoch 2/10
1902/1902 [==============================] - 906s 476ms/step - loss: 1.6368 - average_final_batch_ratio: 0.1667
Epoch 3/10
1902/1902 [==============================] - 915s 481ms/step - loss: 1.6101 - average_final_batch_ratio: 0.1667
Epoch 4/10
1902/1902 [==============================] - 903s 475ms/step - loss: 1.5759 - average_final_batch_ratio: 0.1667
Epoch 5/10
1902/1902 [==============================] - 899s 473ms/step - loss: 1.5477 - average_final_batch_ratio: 0.1667
Epoch 6/10
1902/1902 [==============================] - 900s 473ms/step - loss: 1.5244 - average_final_batch_ratio: 0.1667
Epoch 7/10
1902/1902 [==============================] - 900s 473ms/step - loss: 1.5032 - average_final_batch_ratio: 0.1667
Epoch 8/10
1902/1902 [==============================] - 911s 479ms/step - loss: 1.4877 - average_final_batch_ratio: 0.1667
Epoch 9/10
1902/

In [16]:
total_raw = 0
total_compressed = 0

huffman = Huffman(encoder.vocab_size)
for index, encoded_article in enumerate(articles.articles_generator(1)):
    if index % 1000 == 0:
        article = encoder.decode(encoded_article)
        total_raw += len(article) * 8
        total_compressed += huffman.archive_size(model, encoded_article)
        print('Article %d:\tLength: %d\tAvg Compression: %f' % (index, len(article), total_compressed/total_raw))

Article 0:	Length: 12	Avg Compression: 0.260417
Article 1000:	Length: 38	Avg Compression: 0.272500
Article 2000:	Length: 44	Avg Compression: 0.242021
Article 3000:	Length: 50	Avg Compression: 0.261285
Article 4000:	Length: 56	Avg Compression: 0.263125
Article 5000:	Length: 67	Avg Compression: 0.277154
Article 6000:	Length: 293	Avg Compression: 0.241518
Article 7000:	Length: 880	Avg Compression: 0.235851
Article 8000:	Length: 1498	Avg Compression: 0.240853
Article 9000:	Length: 2197	Avg Compression: 0.242040
Article 10000:	Length: 2988	Avg Compression: 0.240567
Article 11000:	Length: 3898	Avg Compression: 0.229463
Article 12000:	Length: 4954	Avg Compression: 0.236811
Article 13000:	Length: 6351	Avg Compression: 0.235563
Article 14000:	Length: 8043	Avg Compression: 0.240927
Article 15000:	Length: 10150	Avg Compression: 0.244400
Article 16000:	Length: 12843	Avg Compression: 0.237013
Article 17000:	Length: 15922	Avg Compression: 0.231153
Article 18000:	Length: 20350	Avg Compression: 0.2291

# Да опитаме да тренираме върху целия набор от данни

In [19]:
TYPE=np.int16

class Articles:
    EMPTY_ARTICLE = np.array([], dtype=TYPE) # used for padding
    
    def __init__(self, path):
        with open(path, 'rb') as text_file:
            data = text_file.read().decode()

        self.articles = sorted(set(data.split('\0')), key=len)
        self._encoded_articles = None

    @property
    def encoded_articles(self):
        if self._encoded_articles == None:
            self._encoded_articles = [np.array(encoder.encode(article), dtype=TYPE) for article in self.articles]
        
        return self._encoded_articles

    def articles_generator(self, batch_size = 1, start = 0, end = None):
        end = end or len(self.articles)

        for _ in range(batch_size - ((end - start - 1) % batch_size + 1)):
            yield self.EMPTY_ARTICLE

        for article in itertools.islice(self.encoded_articles, start, end):
            yield article

    def subbatch_generator(self, batch_size, batch_length, start = 0, end = None):
        end = end or len(self.articles)

        dataset = tf.data.Dataset.from_generator(self.articles_generator, args=(batch_size, start, end), output_types=TYPE)
        dataset = dataset.padded_batch(batch_size, padded_shapes=([None]), drop_remainder=True)
        dataset = dataset.shuffle(100)

        for batch in dataset.as_numpy_iterator():
            remaining = batch
            while remaining.shape[1] > batch_length + 1:
                yield remaining[:, :batch_length + 1]
                remaining = remaining[:, batch_length:]

            if remaining.shape[1] == batch_length + 1:
                yield remaining
                yield np.zeros((batch_size, batch_length + 1), dtype=TYPE)
            else:
                yield np.hstack([remaining, np.zeros([batch_size, batch_length - remaining.shape[1] + 1])])

    def steps(self, batch_size, batch_length):
        articles = self.articles_generator(batch_size, batch_length)
        return sum(math.ceil(len(article) / batch_length + 1) for i, article in enumerate(articles) if (i + 1) % batch_size == 0)

    def dataset(self, batch_size, batch_length, start = 0, end = None):
        end = end or len(self.articles)

        dataset = tf.data.Dataset.from_generator(self.subbatch_generator, args=(batch_size, batch_length, start, end), output_types=TYPE, output_shapes=(batch_size, batch_length + 1))
        return dataset.map(lambda batch: (batch[:, :-1], batch[:, 1:]))

In [20]:
articles = Articles('page_revisions_text_with_title')

In [21]:
model = Model(articles, './training_checkpoints-27', vocab_size = encoder.vocab_size, embedding_dim=128, rnn_units=512)

In [23]:
model.training_model(256, 256).summary()

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
masking_7 (Masking)          (256, 256)                0         
_________________________________________________________________
embedding_7 (Embedding)      (256, 256, 128)           32640     
_________________________________________________________________
gru_28 (GRU)                 (256, 256, 512)           986112    
_________________________________________________________________
gru_29 (GRU)                 (256, 256, 512)           1575936   
_________________________________________________________________
gru_30 (GRU)                 (256, 256, 512)           1575936   
_________________________________________________________________
gru_31 (GRU)                 (256, 256, 512)           1575936   
_________________________________________________________________
dense_7 (Dense)              (256, 256, 255)          

In [24]:
model.train(256, 256, epochs=50)

Epoch 1/50


12370/12370 [==============================] - 7398s 598ms/step - loss: 1.4139 - average_final_batch_ratio: 0.2623
Epoch 2/50
12370/12370 [==============================] - 5828s 471ms/step - loss: 1.1572 - average_final_batch_ratio: 0.2623
Epoch 3/50
12370/12370 [==============================] - 5917s 478ms/step - loss: 1.1314 - average_final_batch_ratio: 0.2623
Epoch 4/50
12370/12370 [==============================] - 5916s 478ms/step - loss: 1.1127 - average_final_batch_ratio: 0.2623
Epoch 5/50
12370/12370 [==============================] - 5952s 481ms/step - loss: 1.1721 - average_final_batch_ratio: 0.2623
Epoch 6/50
12370/12370 [==============================] - 6054s 489ms/step - loss: nan - average_final_batch_ratio: 0.2623
Epoch 7/50
 4170/12370 [=========>....................] - ETA: 1:01:32 - loss: nan - average_final_batch_ratio: 0.3608

KeyboardInterrupt: 

... не се получава.

# Да опитаме с по-различни слоеве

In [7]:
def loss(labels, logits):
    return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

def average_final_batch_ratio(true_labels, predictions):
    return 0 ** tf.math.abs(true_labels[-1, -1])

class ModelStateResetter(tf.keras.callbacks.Callback):
    def __init__(self):
        pass

    def on_epoch_begin(self, epoch, logs=None):
        self.last_final_batch_count = 0
        
    def on_batch_end(self, batch, logs={}):
        average_final_batch_ratio = logs.get('average_final_batch_ratio', 0)
        final_batch_count = int(round(average_final_batch_ratio * (batch + 1)))
        is_final = final_batch_count - self.last_final_batch_count
        self.last_final_batch_count = final_batch_count
        
        if is_final:
            self.model.reset_states()

class Model:
    def __init__(self, articles, checkpoint_dir, vocab_size, embedding_dim, rnn_units):
        self._articles = articles
        self._batch_size = None
        self._batched_item_length = None
        self._training_model = None
        self._predicting_model = None
        self._vocab_size = vocab_size
        self._embedding_dim = embedding_dim
        self._rnn_units = rnn_units

        self._checkpoint_dir = checkpoint_dir
        self._checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}") # Name of the checkpoint files

    def training_model(self, batch_size, batched_item_length):
        if self._training_model == None or batch_size != self._batch_size or batched_item_length != self._batched_item_length:
            self._batch_size = batch_size
            self._batched_item_length = batched_item_length
            self._training_model = tf.keras.Sequential([
                tf.keras.layers.Masking(mask_value=0, batch_input_shape=[batch_size, batched_item_length]),
                tf.keras.layers.Embedding(self._vocab_size, self._embedding_dim),
                tf.keras.layers.GRU(self._rnn_units, return_sequences=True, stateful=True, recurrent_initializer='glorot_uniform'),
                tf.keras.layers.GRU(self._rnn_units, return_sequences=True, stateful=True, recurrent_initializer='glorot_uniform'),
                tf.keras.layers.Dense(self._vocab_size),
            ])

            if os.path.isdir(self._checkpoint_dir):
                self._training_model.load_weights(tf.train.latest_checkpoint(self._checkpoint_dir))

            self._training_model.compile(optimizer='adam', loss=loss, metrics=[average_final_batch_ratio])
            self._predicting_model = None
        
        return self._training_model

    @property
    def callbacks(self):
        checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(filepath=self._checkpoint_prefix, save_weights_only=True)
        model_state_resetter_callback = ModelStateResetter()
        
        return [checkpoint_callback, model_state_resetter_callback]
    
    def train(self, batch_size, batched_item_length, epochs=1):
        dataset = self._articles.dataset(batch_size, batched_item_length)

        model = self.training_model(batch_size, batched_item_length)

        model.fit(dataset, epochs=epochs, callbacks=self.callbacks)
    
    @property
    def predicting_model(self):
        if self._predicting_model == None:
            self._predicting_model = tf.keras.Sequential([
                tf.keras.layers.Masking(mask_value=0, batch_input_shape=[1, 1]),
                tf.keras.layers.Embedding(self._vocab_size, self._embedding_dim),
                tf.keras.layers.GRU(self._rnn_units, stateful=True, return_sequences=True),
                tf.keras.layers.GRU(self._rnn_units, stateful=True, return_sequences=True),
                tf.keras.layers.Dense(self._vocab_size),
            ])
            
            self._predicting_model.load_weights(tf.train.latest_checkpoint(self._checkpoint_dir))
            self._training_model = None
        
        return self._predicting_model
    
    def predict(self, input_eval):
        return self.predicting_model(input_eval)

In [8]:
model = Model(articles, './training_checkpoints-28', vocab_size = encoder.vocab_size, embedding_dim=128, rnn_units=1024)

In [9]:
model.training_model(256, 192).summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
masking (Masking)            (256, 192)                0         
_________________________________________________________________
embedding (Embedding)        (256, 192, 128)           32640     
_________________________________________________________________
gru (GRU)                    (256, 192, 1024)          3545088   
_________________________________________________________________
gru_1 (GRU)                  (256, 192, 1024)          6297600   
_________________________________________________________________
dense (Dense)                (256, 192, 255)           261375    
Total params: 10,136,703
Trainable params: 10,136,703
Non-trainable params: 0
_________________________________________________________________


In [10]:
model.train(256, 192, epochs=10)

Epoch 1/10
2518/2518 [==============================] - 1538s 611ms/step - loss: 1.9422 - average_final_batch_ratio: 0.1517
Epoch 2/10
2518/2518 [==============================] - 1309s 520ms/step - loss: 1.4509 - average_final_batch_ratio: 0.1517
Epoch 3/10
2518/2518 [==============================] - 1311s 521ms/step - loss: 1.3645 - average_final_batch_ratio: 0.1517
Epoch 4/10
2518/2518 [==============================] - 1299s 516ms/step - loss: 1.3250 - average_final_batch_ratio: 0.1517
Epoch 5/10
2518/2518 [==============================] - 1315s 522ms/step - loss: 1.2911 - average_final_batch_ratio: 0.1517
Epoch 6/10
2518/2518 [==============================] - 1304s 518ms/step - loss: 1.2766 - average_final_batch_ratio: 0.1517
Epoch 7/10
2518/2518 [==============================] - 1305s 518ms/step - loss: 1.2553 - average_final_batch_ratio: 0.1517
Epoch 8/10
2518/2518 [==============================] - 1295s 514ms/step - loss: 1.2448 - average_final_batch_ratio: 0.1517
Epoch 9/

In [12]:
total_raw = 0
total_compressed = 0

huffman = Huffman(encoder.vocab_size)
for index, encoded_article in enumerate(articles.articles_generator(1)):
    if index % 1000 == 0:
        article = encoder.decode(encoded_article)
        total_raw += len(article) * 8
        total_compressed += huffman.archive_size(model, encoded_article)
        print('Article %d:\tLength: %d\tAvg Compression: %f' % (index, len(article), total_compressed/total_raw))

Article 0:	Length: 12	Avg Compression: 0.250000
Article 1000:	Length: 38	Avg Compression: 0.202500
Article 2000:	Length: 44	Avg Compression: 0.166223
Article 3000:	Length: 50	Avg Compression: 0.161458
Article 4000:	Length: 56	Avg Compression: 0.155000
Article 5000:	Length: 67	Avg Compression: 0.173221
Article 6000:	Length: 293	Avg Compression: 0.180580
Article 7000:	Length: 880	Avg Compression: 0.190799
Article 8000:	Length: 1498	Avg Compression: 0.206390
Article 9000:	Length: 2197	Avg Compression: 0.208909
Article 10000:	Length: 2988	Avg Compression: 0.204697
Article 11000:	Length: 3892	Avg Compression: 0.211496
Article 12000:	Length: 4954	Avg Compression: 0.216807
Article 13000:	Length: 6351	Avg Compression: 0.215727
Article 14000:	Length: 8043	Avg Compression: 0.221196
Article 15000:	Length: 10150	Avg Compression: 0.227567
Article 16000:	Length: 12843	Avg Compression: 0.217745
Article 17000:	Length: 15922	Avg Compression: 0.208246
Article 18000:	Length: 20350	Avg Compression: 0.2056

In [13]:
model.train(256, 192, epochs=10)

Epoch 1/10
2518/2518 [==============================] - 1291s 513ms/step - loss: 1.2206 - average_final_batch_ratio: 0.1517
Epoch 2/10
2518/2518 [==============================] - 1299s 516ms/step - loss: 1.2137 - average_final_batch_ratio: 0.1517
Epoch 3/10
2518/2518 [==============================] - 1286s 511ms/step - loss: 1.2112 - average_final_batch_ratio: 0.1517
Epoch 4/10
2518/2518 [==============================] - 1291s 513ms/step - loss: 1.2047 - average_final_batch_ratio: 0.1517
Epoch 5/10
2518/2518 [==============================] - 1283s 510ms/step - loss: 1.2017 - average_final_batch_ratio: 0.1517
Epoch 6/10
2518/2518 [==============================] - 1294s 514ms/step - loss: 1.2009 - average_final_batch_ratio: 0.1517
Epoch 7/10
2518/2518 [==============================] - 1294s 514ms/step - loss: 1.1919 - average_final_batch_ratio: 0.1517
Epoch 8/10
2518/2518 [==============================] - 1291s 513ms/step - loss: 1.1944 - average_final_batch_ratio: 0.1517
Epoch 9/

In [15]:
total_raw = 0
total_compressed = 0

huffman = Huffman(encoder.vocab_size)
for index, encoded_article in enumerate(articles.articles_generator(1)):
    if index % 1000 == 0:
        article = encoder.decode(encoded_article)
        total_raw += len(article) * 8
        total_compressed += huffman.archive_size(model, encoded_article)
        print('Article %d:\tLength: %d\tAvg Compression: %f' % (index, len(article), total_compressed/total_raw))

Article 0:	Length: 12	Avg Compression: 0.239583
Article 1000:	Length: 38	Avg Compression: 0.180000
Article 2000:	Length: 44	Avg Compression: 0.156915
Article 3000:	Length: 50	Avg Compression: 0.151910
Article 4000:	Length: 56	Avg Compression: 0.143750
Article 5000:	Length: 67	Avg Compression: 0.159176
Article 6000:	Length: 293	Avg Compression: 0.176339
Article 7000:	Length: 880	Avg Compression: 0.182118
Article 8000:	Length: 1498	Avg Compression: 0.197626
Article 9000:	Length: 2197	Avg Compression: 0.200730
Article 10000:	Length: 2988	Avg Compression: 0.200465
Article 11000:	Length: 3892	Avg Compression: 0.208448
Article 12000:	Length: 4954	Avg Compression: 0.211739
Article 13000:	Length: 6351	Avg Compression: 0.210045
Article 14000:	Length: 8043	Avg Compression: 0.215214
Article 15000:	Length: 10150	Avg Compression: 0.222105
Article 16000:	Length: 12843	Avg Compression: 0.213049
Article 17000:	Length: 15922	Avg Compression: 0.204296
Article 18000:	Length: 20350	Avg Compression: 0.1998

In [16]:
model.train(256, 192, epochs=10)

Epoch 1/10
2518/2518 [==============================] - 1309s 520ms/step - loss: 1.1853 - average_final_batch_ratio: 0.1517
Epoch 2/10
2518/2518 [==============================] - 1293s 514ms/step - loss: 1.1845 - average_final_batch_ratio: 0.1517
Epoch 3/10
2518/2518 [==============================] - 1291s 513ms/step - loss: 1.1890 - average_final_batch_ratio: 0.1517
Epoch 4/10
2518/2518 [==============================] - 1299s 516ms/step - loss: 1.1765 - average_final_batch_ratio: 0.1517
Epoch 5/10
2518/2518 [==============================] - 1289s 512ms/step - loss: 1.1698 - average_final_batch_ratio: 0.1517
Epoch 6/10
2518/2518 [==============================] - 1292s 513ms/step - loss: 1.1719 - average_final_batch_ratio: 0.1517
Epoch 7/10
2518/2518 [==============================] - 1286s 511ms/step - loss: 1.1703 - average_final_batch_ratio: 0.1517
Epoch 8/10
2518/2518 [==============================] - 1299s 516ms/step - loss: 1.1699 - average_final_batch_ratio: 0.1517
Epoch 9/

In [17]:
total_raw = 0
total_compressed = 0

huffman = Huffman(encoder.vocab_size)
for index, encoded_article in enumerate(articles.articles_generator(1)):
    if index % 1000 == 0:
        article = encoder.decode(encoded_article)
        total_raw += len(article) * 8
        total_compressed += huffman.archive_size(model, encoded_article)
        print('Article %d:\tLength: %d\tAvg Compression: %f' % (index, len(article), total_compressed/total_raw))

Article 0:	Length: 12	Avg Compression: 0.250000
Article 1000:	Length: 38	Avg Compression: 0.215000
Article 2000:	Length: 44	Avg Compression: 0.180851
Article 3000:	Length: 50	Avg Compression: 0.167535
Article 4000:	Length: 56	Avg Compression: 0.159375
Article 5000:	Length: 67	Avg Compression: 0.179307
Article 6000:	Length: 293	Avg Compression: 0.186161
Article 7000:	Length: 880	Avg Compression: 0.190278
Article 8000:	Length: 1498	Avg Compression: 0.200774
Article 9000:	Length: 2197	Avg Compression: 0.198174
Article 10000:	Length: 2988	Avg Compression: 0.195356
Article 11000:	Length: 3892	Avg Compression: 0.204224
Article 12000:	Length: 4954	Avg Compression: 0.208778
Article 13000:	Length: 6351	Avg Compression: 0.206196
Article 14000:	Length: 8043	Avg Compression: 0.211591
Article 15000:	Length: 10150	Avg Compression: 0.216429
Article 16000:	Length: 12843	Avg Compression: 0.207696
Article 17000:	Length: 15922	Avg Compression: 0.198601
Article 18000:	Length: 20350	Avg Compression: 0.1965

In [18]:
model.train(256, 192, epochs=10)

Epoch 1/10


2518/2518 [==============================] - 1304s 518ms/step - loss: 1.1744 - average_final_batch_ratio: 0.1517
Epoch 2/10
2518/2518 [==============================] - 1307s 519ms/step - loss: 1.1705 - average_final_batch_ratio: 0.1517
Epoch 3/10
2518/2518 [==============================] - 1296s 515ms/step - loss: 1.1676 - average_final_batch_ratio: 0.1517
Epoch 4/10
2518/2518 [==============================] - 1300s 516ms/step - loss: 1.1664 - average_final_batch_ratio: 0.1517
Epoch 5/10
2518/2518 [==============================] - 1289s 512ms/step - loss: 1.1691 - average_final_batch_ratio: 0.1517
Epoch 6/10
2518/2518 [==============================] - 1311s 521ms/step - loss: 1.1658 - average_final_batch_ratio: 0.1517
Epoch 7/10
2518/2518 [==============================] - 1291s 513ms/step - loss: 1.2481 - average_final_batch_ratio: 0.1517
Epoch 8/10
2518/2518 [==============================] - 1325s 526ms/step - loss: 1.1746 - average_final_batch_ratio: 0.1517
Epoch 9/10
2518/251

In [19]:
total_raw = 0
total_compressed = 0

huffman = Huffman(encoder.vocab_size)
for index, encoded_article in enumerate(articles.articles_generator(1)):
    if index % 1000 == 0:
        article = encoder.decode(encoded_article)
        total_raw += len(article) * 8
        total_compressed += huffman.archive_size(model, encoded_article)
        print('Article %d:\tLength: %d\tAvg Compression: %f' % (index, len(article), total_compressed/total_raw))

Article 0:	Length: 12	Avg Compression: 0.229167
Article 1000:	Length: 38	Avg Compression: 0.180000
Article 2000:	Length: 44	Avg Compression: 0.162234
Article 3000:	Length: 50	Avg Compression: 0.158854
Article 4000:	Length: 56	Avg Compression: 0.151875
Article 5000:	Length: 67	Avg Compression: 0.170412
Article 6000:	Length: 293	Avg Compression: 0.180580
Article 7000:	Length: 880	Avg Compression: 0.185764
Article 8000:	Length: 1498	Avg Compression: 0.194733
Article 9000:	Length: 2197	Avg Compression: 0.194085
Article 10000:	Length: 2988	Avg Compression: 0.193155
Article 11000:	Length: 3892	Avg Compression: 0.201332
Article 12000:	Length: 4954	Avg Compression: 0.205529
Article 13000:	Length: 6351	Avg Compression: 0.203366
Article 14000:	Length: 8043	Avg Compression: 0.209618
Article 15000:	Length: 10150	Avg Compression: 0.214728
Article 16000:	Length: 12843	Avg Compression: 0.206700
Article 17000:	Length: 15922	Avg Compression: 0.197307
Article 18000:	Length: 20350	Avg Compression: 0.1931

In [20]:
model.train(256, 192, epochs=5)

Epoch 1/5
2518/2518 [==============================] - 1330s 528ms/step - loss: 1.1618 - average_final_batch_ratio: 0.1517
Epoch 2/5
2518/2518 [==============================] - 1332s 529ms/step - loss: 1.1628 - average_final_batch_ratio: 0.1517
Epoch 3/5
2518/2518 [==============================] - 1326s 527ms/step - loss: 1.1652 - average_final_batch_ratio: 0.1517
Epoch 4/5
2518/2518 [==============================] - 1339s 532ms/step - loss: 1.1603 - average_final_batch_ratio: 0.1517
Epoch 5/5
2518/2518 [==============================] - 1336s 531ms/step - loss: 1.1621 - average_final_batch_ratio: 0.1517
